In [11]:
import boto3
import json
import time

In [2]:
lex = boto3.client('lex-models', region_name='us-east-1')
lambd = boto3.client('lambda', region_name='us-east-1')
iam = boto3.client('iam', region_name='us-east-1')


In [3]:
import sagemaker
from sagemaker import get_execution_role

aws_role = get_execution_role()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [13]:
aws_role

'arn:aws:iam::287758680514:role/service-role/AmazonSageMaker-ExecutionRole-20231027T004238'

In [14]:
def wait(secs):
    time.sleep(secs)

In [7]:
role_policy_document = {
        u'Version': u'2012-10-17',
        u'Statement': [
            {u'Effect': u'Allow', u'Principal': {u'Service': [u'lambda.amazonaws.com']},
             u'Action': [u'sts:AssumeRole']},
        ]
    }

In [8]:
role_policy_document

{'Version': '2012-10-17',
 'Statement': [{'Effect': 'Allow',
   'Principal': {'Service': ['lambda.amazonaws.com']},
   'Action': ['sts:AssumeRole']}]}

In [9]:
role = iam.create_role(RoleName='shakirachatbot-role', AssumeRolePolicyDocument=json.dumps(role_policy_document))['Role']
print ('Created custom role shakirachatbot-role')
wait(3)

ClientError: An error occurred (AccessDenied) when calling the CreateRole operation: User: arn:aws:sts::287758680514:assumed-role/AmazonSageMaker-ExecutionRole-20231027T004238/SageMaker is not authorized to perform: iam:CreateRole on resource: arn:aws:iam::287758680514:role/shakirachatbot-role because no identity-based policy allows the iam:CreateRole action

In [4]:
inline_policy = {
    u'Version': u'2012-10-17',
    u'Statement': [
        {
            u'Effect': u'Allow',
            u'Action': [
                u'polly:SynthesizeSpeech',
                u's3:ListBucket',
                u's3:PutObject',
                u's3:PutObjectAcl',
                u'logs:CreateLogGroup',
                u'logs:CreateLogStream',
                u'logs:PutLogEvents'
            ],
            u'Resource': u'*'
        }
    ]
}


In [5]:
inline_policy

{'Version': '2012-10-17',
 'Statement': [{'Effect': 'Allow',
   'Action': ['polly:SynthesizeSpeech',
    's3:ListBucket',
    's3:PutObject',
    's3:PutObjectAcl',
    'logs:CreateLogGroup',
    'logs:CreateLogStream',
    'logs:PutLogEvents'],
   'Resource': '*'}]}

In [13]:
iam.put_role_policy(RoleName='AmazonSageMaker-ExecutionRole-20231027T004238', PolicyName='BotAccess', PolicyDocument=json.dumps(inline_policy))
print ('Created inline policy for role shakirachatbot-role')


Created inline policy for role shakirachatbot-role


In [15]:
lex = boto3.client('lex-models', region_name='us-east-1')
lambd = boto3.client('lambda', region_name='us-east-1')
iam = boto3.client('iam', region_name='us-east-1')

In [19]:
from zipfile import ZipFile

with ZipFile('lambda-package.zip', 'w') as myzip:
    myzip.write('chatbot/__init__.py')
    myzip.write('chatbot/botcontrol.py')

FileNotFoundError: [Errno 2] No such file or directory: 'chatbot/__init__.py'

In [18]:
botcontrol_function = {
    u'Code': {u'ZipFile': open('lambda-package.zip').read()},
    u'Description': u'Function controling the ShakiraChatbot, powered up by Lex',
    u'Environment': {u'Variables': {u'MASHAPE_API_KEY': os.environ['MASHAPE_API_KEY']}},
    u'FunctionName': u'{}'.format(FUNCTION_NAME),
    u'Handler': u'chatbot/botcontrol.lambda_handler',
    u'MemorySize': 128,
    u'Role': u'arn:aws:iam::{}:role/shakirachatbot-role'.format(ACCOUNT_ID),
    u'Runtime': u'python2.7',
    u'Timeout': 10,
    u'TracingConfig': {u'Mode': u'PassThrough'},
    u'Publish': True,
}

FileNotFoundError: [Errno 2] No such file or directory: 'lambda-package.zip'

In [17]:
lambd.create_function(**botcontrol_function)
print ('Created lambda function {}'.format(FUNCTION_NAME))
wait(3)

NameError: name 'botcontrol_function' is not defined